# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
#прописываем нужные нам библиотеки 
import pandas as pd
import math
import numpy as np
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
def fun_date(data):
    print('**Общая информация датафрейма**')
    print(data.info())
    print('-------------------------------------------------------------------------------------------------------------')
    print('**Просмотр статистических данных**')
    print(data.describe())
    print('-------------------------------------------------------------------------------------------------------------')
    print('**Вывод рандомных строк датафрейма**')
    print(data.sample(n=3, random_state=1))
    print('-------------------------------------------------------------------------------------------------------------')
    print('**Вывод первых пяти строк датафрейма**')
    print(data.head())
    print('-------------------------------------------------------------------------------------------------------------')
    print('**Вывод количесва дубликатов датафрейма**')
    print(data.duplicated().sum())
    print('-------------------------------------------------------------------------------------------------------------')
    print('**Вывод количесва пропущеных значений датафрейма**')
    print(data.isnull().sum())
    print('-------------------------------------------------------------------------------------------------------------')
print(fun_date(df))

**Общая информация датафрейма**
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None
-------------------------------------------------------------------------------------------------------------
**Просмотр статистических данных**
             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   234.569872    36.148326   7570.968246     0.461100
min       0.000000     0.000000     0.000000      0.000000     0.000000
25%      40.000000   274.575000     9.000000  12491.902500     0.000000
50%      62.000000   430.600000    30.000000  16943.235000  

Все данные в порядке, можем работать!

## Разбейте данные на выборки

In [5]:
features = df.drop(['is_ultra', 'minutes'], axis =1)
target = df['is_ultra']


features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=12345)
features_test, features_valid, target_test, target_valid = train_test_split(features_test, target_test, test_size=0.5, random_state=12345)

print('Обучающая выборка:',features_train.shape)
print(target_train.shape)
print('Валидационная выборка:',features_valid.shape)
print(target_valid.shape)
print('Тестовая выборка:',features_test.shape)
print(target_test.shape)

Обучающая выборка: (1928, 3)
(1928,)
Валидационная выборка: (643, 3)
(643,)
Тестовая выборка: (643, 3)
(643,)


Разбили нашу выборку в соотношении 3:1:1

## Исследуйте модели

In [6]:
#Дерево решений
for depth in range(1, 6):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
   
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid)) 

max_depth = 1 : 0.7356143079315708
max_depth = 2 : 0.7698289269051322
max_depth = 3 : 0.7744945567651633
max_depth = 4 : 0.7729393468118196
max_depth = 5 : 0.7838258164852255


Наилочная точность выборки оказалась 0.78, при глубине дерева 5

In [7]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train,target_train) 
    result = model.score(features_valid,target_valid) 
    if result > best_result:
        best_model = model 
        best_result = result 

print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.7869362363919129


In [8]:
model = LogisticRegression(random_state=12345)# инициализируйте модель логистической регрессии с параметром random_state=12345
model.fit(features_train,target_train) # обучите модель на тренировочной выборке
result = model.score(features_valid,target_valid) # получите метрику качества модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7402799377916018


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Пока что можно сделать вывод, что лучше всего себя показывает метод "дерево решений"

## Проверьте модель на тестовой выборке

In [9]:
#Дерево решений
model_dt=DecisionTreeClassifier(random_state=12345, max_depth=5, min_samples_leaf=1, min_samples_split= 3)
model_dt.fit(features_train, target_train)
accuracy_dt=model_dt.score(features_test, target_test)
print(accuracy_dt)

0.7900466562986003


In [10]:
#Случайный лес
model_dt=RandomForestClassifier(random_state=12345, max_depth=10, min_samples_leaf=1, min_samples_split= 3)
model_dt.fit(features_train, target_train)
accuracy_dt=model_dt.score(features_test, target_test)
print(accuracy_dt)


0.7791601866251944


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [11]:
#логическая регрессия
model_dt=LogisticRegression(random_state=12345)
model_dt.fit(features_train, target_train)
accuracy_dt=model_dt.score(features_test, target_test)
print(accuracy_dt)

0.7511664074650077


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Лучше всего себя показал метод "дерево решений" с точность 0.79 и высокой скоростью, в отличии от остальных методов.

## (бонус) Проверьте модели на адекватность

In [12]:
a=np.random.randint(0, 2, size=len(target_test))
def error_count(answers, predictions):
    return ((answers == predictions).sum())/len(answers)
print(error_count(a, target_test))

0.5007776049766719


Из полученный данных предположу, что модель вполне адекватная